### 1、缓存数据设计

* 需求：利用redis数据库，读写效率更高，存储mysql中的数据，避免频繁读写mysql，提高项目数据访问的效率；

* 如何设计数据类型：经常需要查询的数据选择存入redis中，

  * 1、用户资料设计，所有用户资料都要存入redis中；
  * 2、一个key，存储所有数据；单独的数据无法维护有效期；user1过期，user2可能会存在，不能删除数据；

  ~~~python
  # string/list/hash/set/zset
  str：users = 'user1-mobile-name-intro,user2,user3' # 操作不方便
  list = [user1,user2,user3] # 操作不方便
  hash = {
    user1:数据，
    user2:数据,
    user3:数据
  } # 可以
  set和zset操作数据不方便，
  zset = (member1,score1,memger2,score2)
  zset (id1 user1,id2 user2) # 可以
  ~~~

  * 3、一个key，存储一条数据；每条数据都有自己的记录，有自己的有效期

  ~~~python
  # string/list/hash/set/zset
  str：
  user:1:info = 'mobile:name:xxx' # 操作方便,一旦用户修改数据，只需要删除即可
  user:2:info = 'mobile:name:xxx'    
  list:
  user:1:info = [mobile,name,profile_photo...]
  user:1:info = {
    user1:数据，
    user2:数据,
    user3:数据
  } # 可以，大材小用
  set和zset操作数据不方便，
  
  # 可以，大材小用
  user:1:info = (score1,memger1,score2,member2)
  zset (id1 user1,id2 user2) 
  
  ~~~

  

### 2、持久存储

* 统计数据：用户关注、用户发布文章、搜索历史等；

* 如何设计数据类型？

  * 需求：用户发布文章
  * 1、一个key，存储所有数据；

  ~~~python
  # string/list/hash/set/zset
  key = 'user:count:articles'
  +---------+-------------------+
  | user_id | count(article_id) |
  +---------+-------------------+
  |       1 |             46145 |
  |       2 |             46357 |
  |       3 |             46187 |
  |       5 |                25 |
  +---------+-------------------+
  string：'user:count:articles' = '1:46145,2:46357...' # 操作数据不方便
  hash:'user:count:articles' = {user1:46145,user2:46357...} # 可以
  zset:'user:count:articles' = (score1,member1,46145,user1,46347,user2...) # 可以，建议使用，可以排序
  ~~~

  

  * 2、一个key，存储一条数据；

  ~~~python
  # string/list/hash/set/zset
  key = 'user1:count:articles'
  +---------+-------------------+
  | user_id | count(article_id) |
  +---------+-------------------+
  |       1 |             46145 |
  |       2 |             46357 |
  |       3 |             46187 |
  |       5 |                25 |
  +---------+-------------------+
  string：'user1:count:articles' = '1:46145...' # 可以存储，不能排序
  hash:'user1:count:articles' = {user1:46145,user2:46357...} # 可以
  zset:'user1:count:articles' = (score1,member1,46145,user1,46347,user2...) # 可以，建议使用，可以排序
  
  
  ~~~

  * 持久存储：利用redis-master和slave存储相关统计数据，选择zset；
  * 随着项目的运行，数据有可能会产生redis和mysql的不一致，可以通过定时任务定期修正数据。

### 3、定时任务apscheduler

* 触发器：
  * 1、date特点：在特定的时间点，只执行一次；
  * 2、interval特点：固定时间间隔，可以执行多次；
  * 3、cron特点：特定时间周期性触发，可以执行多次；每次任务执行的间隔时间不固定；

